In [ ]:
import os

! git clone https://github.com/fengredrum/cnn-xla.git
os.chdir('cnn-xla')

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, optim

from models import alexnet, resnet18, densenet18
from utils import load_data_cifar_10, train_model

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(torch.cuda.get_device_name(0))
else:
    device = torch.device('cpu')

In [ ]:
model_name = "alexnet"  #@param ["alexnet", "resnet", "densenet"]

activation = "mish"  #@param ["relu", "swish", "mish"]

batch_size = 256  #@param {type:"integer"}

lr = 0.01  #@param {type:"number"}

num_epochs = 20  #@param {type:"integer"}

if model_name == 'alexnet':
    net = alexnet(activation='mish')
elif model_name == 'resnet':
    net = resnet18([2, 2, 2, 2], activation='mish')
elif model_name == 'densenet':
    net = densenet18([4, 4, 4, 4], activation='mish')
else:
    raise NotImplementedError

In [ ]:
train_iter, test_iter = load_data_cifar_10(batch_size)
optimizer = optim.Adam(net.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
train_model(net, train_iter, test_iter, batch_size, optimizer, scheduler,
            device, num_epochs)

In [ ]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir runs/